#### 다나와 크롤링

##### 다나와 무선청소기 크롤링

- 무선 청소기 URL : https://search.danawa.com/dsearch.php?query=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&page=1&limit=120

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.autonotebook import tqdm

In [2]:
# tqdm progressbar 워닝때문 설치
!pip install ipywidgets

                                              0.0/138.3 kB ? eta -:--:--
     -------------------------------------- 138.3/138.3 kB 4.1 MB/s eta 0:00:00
                                              0.0/2.3 MB ? eta -:--:--
     ----------                               0.6/2.3 MB 12.2 MB/s eta 0:00:01
     ---------------                          0.9/2.3 MB 11.4 MB/s eta 0:00:01
     ----------------------                   1.3/2.3 MB 9.2 MB/s eta 0:00:01
     --------------------------------------   2.2/2.3 MB 11.8 MB/s eta 0:00:01
     ---------------------------------------- 2.3/2.3 MB 11.2 MB/s eta 0:00:00
                                              0.0/215.0 kB ? eta -:--:--
     ---------------------------------------- 215.0/215.0 kB ? eta 0:00:00


In [4]:
# 크롬 드라이버로 크롬 브라우저 실행
driver = webdriver.Chrome()
url = 'https://search.danawa.com/dsearch.php?query=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&page=1&limit=120'
driver.get(url)

In [8]:
# bs 사용 html 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
soup

<html class="js no-touch" lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml"><head>
<title>무선청소기 : 다나와 통합검색</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="무선청소기 통합검색 : 다나와 통합검색" name="title"/>
<meta content="'무선청소기'의 다나와 통합검색 결과입니다." name="description"/>
<meta content="무선청소기, 다나와, 최저가, 가격비교, 검색, 통합검색, 차이슨 무선청소기, 샤오미 무선청소기, 삼성 무선청소기, 다이슨 무선청소기, 일렉트로룩스 무선청소기, lg 무선청소기, 디베아 무선청소기, 차량용 무선청소기, 유선청소기, 진공청소기" name="keywords"/>
<meta content="무선청소기 통합검색 : 다나와 통합검색" property="og:title"/>
<meta content="'무선청소기'의 다나와 통합검색 결과입니다." property="og:description"/>
<meta content="https://img.danawa.com/new/tour/img/logo/sns_danawa.jpg" property="og:image"/>
<meta content="width=1280" name="viewport"/>
<link href="//img.danawa.com/new/danawa_main/v1/img/danawa_favicon.ico" rel="shortcut icon"/>
<link href="//static.danawa.com/css/common.css?v=2305031517" rel="stylesheet" type="text/css"/>
<link href

In [144]:
# 제품리스트만
prod_items = soup.select('li.prod_item')
len(prod_items)

123

In [45]:
# 첫번째 제품으로 값 가져오기 체크
# 1. 제품명 가져오기
# strip() - 공백, \n 다 없애줌 / 중간에 애드스마트 광고 부분 예외처리 제거대상
title = prod_items[0].select('p.prod_name')[0].text.strip()
title

'삼성전자 비스포크 제트 VS20B956D5'

In [55]:
# 2. 제품 스펙 가져오기
spec_list =prod_items[0].select('div.spec_list')[0].text.strip().replace('\t','')
spec_list

'핸디스틱청소기 / 무선 / 흡입+물걸레겸용 / 흡입력: 220W / [구성] 거치대: 일체형청정스테이션 / 바닥 / 물걸레 / 솔형 / 틈새 / 연장툴 / [배터리] 분리형 (1개) / 충전시간: 3시간30분 / 사용시간: 1시간 / [성능] 디지털인버터모터 / 5단계여과 / [기능] IoT기능 / 디스플레이표시창 / 자동물공급 / LED라이트 / 배터리잔량표시 / 자동먼지비움 / [부가] 먼지통용량: 0.5L / 색상: 산토리니베이지 / 2022년형 / 물걸레 / 크기(가로x세로x깊이): 250x930x202mm'

In [84]:
# 3. 가격 정보
price = prod_items[0].select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','')
price

'555480'

In [98]:
# 애드스마트 광고영역인지 판별
'ad_section' in prod_items[11].select('div')[0]['class']

True

##### 반복문으로 검색결과 상품 정보 리스트 추출

In [102]:
prod_data = []
count = 0
for prod_item in prod_items:
    # if count >= 7: break

    if 'ad_section' in prod_item.select('div')[0]['class']:
        continue # 애드스마트 영역이면 패스
    try:
        # 1. 제품명
        title = prod_item.select('p.prod_name')[0].text.strip()
        # 2. 제품스펙
        spec_list = prod_item.select('div.spec_list')[0].text.strip().replace('\t','')
        # 3. 제품가격
        price = prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','')
        prod_data.append([title, spec_list, price])
    except:
        pass

    # count += 1 

In [107]:
pd.DataFrame(prod_data).head()

,0,1,2
0,삼성전자 비스포크 제트 VS20B956D5,핸디스틱청소기 / 무선 / 흡입+물걸레겸용 / 흡입력: 220W / [구성] 거치대...,555480
1,LG전자 오브제컬렉션 코드제로 A9S AX9984,핸디스틱청소기 / 무선 / 흡입+물걸레겸용 / 흡입력: 250W / [구성] 거치대...,1136630
2,LG전자 오브제컬렉션 코드제로 AS9202WD,핸디스틱청소기 / 무선 / 흡입전용 / 흡입력: 210W / [구성] 거치대: 거치...,530620
3,샤오미 드리미 V11,핸디스틱청소기 / 무선 / 흡입전용 / 흡입력: 150AW / 흡입력: 25000P...,179000
4,삼성전자 비스포크 제트 VS20A956A3,핸디스틱청소기 / 무선 / 흡입전용 / 흡입력: 210W / [구성] 거치대: 일체...,482660


In [108]:
# 함수 만들기
def get_prod_items(prod_items):
    prod_data = []

    for prod_item in prod_items:
        # if count >= 7: break

        if 'ad_section' in prod_item.select('div')[0]['class']:
            continue # 애드스마트 영역이면 패스
        try:
            # 1. 제품명
            title = prod_item.select('p.prod_name')[0].text.strip()
            # 2. 제품스펙
            spec_list = prod_item.select('div.spec_list')[0].text.strip().replace('\t','')
            # 3. 제품가격
            price = prod_item.select('li.rank_one > p.price_sect > a > strong')[0].text.strip().replace(',','')
            prod_data.append([title, spec_list, price])
        except:
            pass
        
    return prod_data

In [118]:
# 함수 테스트
prod_items1 = soup.select('li.prod_item')
prod_data1 = get_prod_items(prod_items1)
pd.DataFrame(prod_data1).tail()

,0,1,2
115,일렉트로룩스 ZB6218STM,핸디청소기 / 무선 / 흡입전용 / [구성] 거치대: 충전+거치 / [배터리] 내장...,147150
116,한빛시스템 HV-5099,핸디스틱청소기 / 무선 / 흡입전용 / [구성] 바닥 / 솔형 / 틈새 / [배터리...,75090
117,캐치웰 CX PRO,핸디스틱청소기 / 무선 / 흡입전용 / [구성] 바닥 / 침구 / 솔형 / 틈새 /...,329000
118,LG전자 코드제로 A9S AS9271,핸디스틱청소기 / 무선 / 흡입+물걸레겸용 / 흡입력: 210W / [구성] 거치대...,677310
119,리하스 S3,핸디스틱청소기 / 무선 / 흡입전용 / 흡입력: 30000Pa / [구성] 거치대:...,209000


##### 페이지 변경하면서 크롤링

In [119]:
# 페이지 URL 생성함수
def get_search_page_url(keyword, page):
    return f'https://search.danawa.com/dsearch.php?query={keyword}&page={page}&limit=120&sort=saveDESC'

In [124]:
# 함수 테스트
keyword = '무선청소기'
page = 150
url = get_search_page_url(keyword, page)
driver.get(url)

##### 여러페이지 상품 데이터 수집

In [147]:
driver = webdriver.Chrome()
# 암묵적으로 웹 자원 로드를 위해서 3초 대기
# driver.implicitly_wait(5)

In [152]:
from time import sleep

total_page = 20
prod_total_data = []

for page in tqdm(range(1, total_page+1)):
    url = get_search_page_url(keyword='TV', page=page)
    driver.get(url)
    sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    prod_items = soup.select('li.prod_item')
    prod_item_list = get_prod_items(prod_items)

    prod_total_data += prod_item_list
    



100%|██████████| 20/20 [05:26<00:00, 16.35s/it]


In [153]:
pd.DataFrame(prod_total_data)

,0,1,2
0,LG전자 스탠바이미 Go 27LX5QKNA,LED TV / 27인치(68cm) / LED / FHD / 60Hz / 인공지능화...,1096200
1,LG전자 스탠바이미 27ART10AKPL,LED TV / 27인치(69cm) / LED / FHD / IPS패널계열 / 60...,890100
2,삼성전자 Crystal UHD KU50UA7050FXKR,LED TV / 50인치(125cm) / LED / 4K UHD / 60Hz / 4...,499380
3,삼성전자 QLED KQ65QC60AFXKR,QLED TV / 65인치(163cm) / QLED / 4K UHD / VA패널계열...,1059210
4,LG전자 울트라HD 65UR642S0NC,LED TV / 65인치(163cm) / LED / 4K UHD / 60Hz / 4...,843310
...,...,...,...
2395,디엘티 ZEUS 아르떼 D3255H,LED TV / 32인치(81cm) / LED / HD / 60Hz / 명암비: 5...,135600
2396,삼성전자 LH65QHBEBGCXKR,실내용 사이니지 / 디스플레이모듈 / LED / 65인치(164cm) / 4K UH...,1349950
2397,와사비망고 QG551 QLED 와글와글 플러스,QLED TV / 55인치(138cm) / QLED / 4K UHD / VA패널계열...,439000
2398,삼성전자 LH32DCE2LGA,실내용 사이니지 / 디스플레이모듈 / LED / 32인치(80cm) / FHD / ...,439000


In [155]:
# 수집 데이터 저장
df = pd.DataFrame(prod_total_data)
df.columns = ['상품명', '스펙목록', '가격']
df.to_excel('./2_다나와_TV크롤링결과.xlsx', index=False)